In [2]:
# ! pip install langchain
# ! pip install openai

In [3]:
# ! pip install unstructured
# ! pip install libmagic
# ! pip install langchain

In [1]:
from apikey import OPEN_AI_KEY
from langchain.llms import OpenAI

In [3]:
import os 
# add your openai api key
os.environ['OPENAI_API_KEY'] = "OPEN_AI_KEY"

In [4]:
from langchain.document_loaders import UnstructuredFileLoader

# load documents you want to llm to answer from.
loaders = UnstructuredFileLoader(r"D:\AdVantage Digital\Our Services.docx")
data = loaders.load()

In [5]:
data

[Document(page_content="Our Services\n\nAI SERVICES\n\nAI-Powered Chatbots: Increasing Customer Engagement\n\nAdVantage AI specializes in creating AI-powered chatbots that change how businesses interact with their customers. These modern virtual assistants are designed to speak in a natural and seamless manner, responding to user requests promptly and accurately.\n\nHow AI-Based Chatbots Work:\n\nTo understand and analyze user inputs, our AI-powered chatbots use the power of Artificial Intelligence and Natural Language Processing (NLP). They are trained on massive datasets, allowing them to understand human language characteristics and reply in a human-like manner. This implies that interacting with our chatbot is just like interacting with a real person!\n\nCustomized for your Company:\n\nWe understand that each business is unique, which is why our AI-powered chatbots can be easily customized. We design the chatbot for your specific needs and industry, whether you need it for customer

In [6]:
from langchain.text_splitter import CharacterTextSplitter

# This splits the data into chunks 
text_splitter = CharacterTextSplitter(separator = '\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)

docs = text_splitter.split_documents(data)

In [7]:
len(docs)

6

In [8]:
import pickle
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [16]:
persist_directory = 'adv-db'

# Create a vector database using chroma
vectordb = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory=persist_directory)


In [17]:
# persist the database 
vectordb.persist()
vectordb = None

In [18]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [19]:
# retriever to perform semantic search
retriever = vectordb.as_retriever()

In [20]:
from langchain.chains import RetrievalQA

# create a chain to retrieve the info and print the answer for the query
chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                    chain_type='stuff',
                                    retriever=retriever,
                                    return_source_documents=True)


In [21]:
def process_llm_response(llm_response):
    print(llm_response['result'])

In [22]:
query = 'Services offered?'
llm_response = chain(query)
print(process_llm_response(llm_response))

 AdVantage AI offers software development services that revolve around user interface (UI) and user experience (UX). We also specialize in creating AI-powered chatbots that use the power of Artificial Intelligence and Natural Language Processing (NLP) to understand and analyze user inputs. Our services are focused on providing responsive and high-performing products that leave a lasting impression on your audience and are tailored to fit the specific needs of your company.
None


In [23]:
query1 = "What are the AI Automation services?"
llm_response1 = chain(query1)
print(process_llm_response(llm_response1))

 AdVantage AI provides AI-Powered Chatbots, Knowledge-Based Automated Chatbots, and General Chatbots. They specialize in creating AI-powered chatbots that allow businesses to interact with their customers in a natural and seamless manner. They also offer software development services to create responsive and high-performing applications.
None
